# Project Final Report
#### By: Mohak Batra, Ellina Hao, Sophia Margarita Mokretsova

Title
Introduction:
provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your report
clearly state the question you tried to answer with your project
identify and fully describe the dataset that was used to answer the question


Methods & Results:
describe the methods you used to perform your analysis from beginning to end that narrates the analysis code.
your report should include code which:
loads data 
wrangles and cleans the data to the format necessary for the planned analysis
performs a summary of the data set that is relevant for exploratory data analysis related to the planned analysis 
creates a visualization of the dataset that is relevant for exploratory data analysis related to the planned analysis
performs the data analysis
creates a visualization of the analysis 


note: all figures should have a figure number and a legend
Discussion:
summarize what you found
discuss whether this is what you expected to find
discuss what impact could such findings have
discuss what future questions could this lead to
References
You may include references if necessary, as long as they all have a consistent citation style.

# Introduction: 
### Predicting Game Related Newsletter Subscription from Player Behavior
Our project takes a closer look at if we can predict if a Minecraft player is subscribed to a newsletter based on their in game behavior. We looked at the player's age, session length, and total playtime in order to explore our prediction. We have been provided two datasets, players.csv with basic player information, and sessions.csv with recorded gameplay sessions. 

# Method Used:
### KNN Classification 

We first started our project by cleaning up and wrangling our data into the format necessary for our planned analysis. 

After preparing the data, we made a K-Nearest Neighbours (KNN) classification model. We trained the model using age, average session length, and total playtime to see if it could predict whether a player was subscribed or not to a game related newsletter. 

In [7]:
# Importing Libraries
library(tidyverse)
library(repr)
library(tidymodels)


In [8]:
# Raw dataset URLs from github imported files
player_data_url <- "https://raw.githubusercontent.com/MohakB3/dsci100-project/refs/heads/main/data/players.csv"
sessions_data_url <- "https://raw.githubusercontent.com/MohakB3/dsci100-project/refs/heads/main/data/sessions.csv"

# Reading Data (with the column specification message disabled) and Omitting NAs
player_data <- na.omit(read_csv(player_data_url, show_col_types = FALSE))
sessions_data <- na.omit(read_csv(sessions_data_url, show_col_types = FALSE))

# Wrangling Sessions Data 
sessions_wrangled_data <- sessions_data |>
    #creating new column session_length, converting start_time and end_time into date-time format and subtracting them. This gives us how long each session was.
    mutate(session_length = as.POSIXct(end_time,format = "%d/%m/%Y %H:%M") - as.POSIXct(start_time, format = "%d/%m/%Y %H:%M")) |> 
    #selecting and keeping only two columns
    select(hashedEmail, session_length) |> 
    #grouping all sessions belonging to the same hashedEmail together
    group_by(hashedEmail) |>
    #calculating each player’s average session length
    summarize(player_average_session_length = as.numeric(mean(session_length)))


# Merging Player Data and Wrangled Sessions Data
merged_data <- left_join(sessions_wrangled_data, player_data, by = "hashedEmail") |> na.omit()

# Calculating Mean Values for all Quantitative Variables
mean_quantitative_data <- merged_data |>
    summarize(mean_age = mean(Age), mean_played_hours = mean(played_hours), mean_session_length = mean(player_average_session_length))
mean_quantitative_data

mean_age,mean_played_hours,mean_session_length
<dbl>,<dbl>,<dbl>
21.58537,9.313008,32.85302


# Discussion:
